In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import stac

In [2]:
# Assumes a formattting where rows are datasets, and the methods are columns

df = pd.read_excel('Data/6.FINAL.xlsx',sheet_name='Accuracy', index_col=0)


df =  df.iloc[:-2,:]
data = np.asarray(df)

print("Read data")
# To be safe, ensure this matches what was expected
num_datasets, num_methods = df.shape
print("Methods:", num_methods, "Datasets:", num_datasets)

Read data
Methods: 5 Datasets: 10


In [3]:
alpha = 0.05 # Set this to the desired alpha/signifance level

stat, p = stats.friedmanchisquare(*data)

reject = p <= alpha
print("Should we reject H0 (i.e. is there a difference in the means) at the", (1-alpha)*100, "% confidence level?", reject)

Should we reject H0 (i.e. is there a difference in the means) at the 95.0 % confidence level? True


In [4]:
p

0.018817028401625756

In [5]:
stat

19.856796116504864

In [6]:
if not reject:
    raise Exception("Exiting early. The rankings are only relevant if there was a difference in the means i.e. if we rejected h0 above")

In [7]:
statistic, p_value, ranking, rank_cmp  = stac.friedman_test(*np.transpose(data))
ranks = {key: rank_cmp[i] for i, key in enumerate(list(df.columns))}

comparisons, z_values, p_values, adj_p_values = stac.holm_test(ranks, control="proposed")
adj_p_values = np.asarray(adj_p_values)

In [8]:
adj_p_values,z_values

(array([6.16690317e-08, 3.01865766e-04, 2.92543317e-03, 1.57299207e-01]),
 [5.65685424949238, 3.889087296526011, 3.1819805153394634, 1.414213562373095])

In [9]:
for method, rank in ranks.items():
    print(method +":", "%.2f" % rank)

gnb: 3.89
svc: 1.41
rfc: 5.66
xgb: 3.18
proposed: 7.07


In [10]:
holm_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < alpha}, index=comparisons)

In [11]:
alpha

0.05

In [12]:
holm_scores

,p,sig
proposed vs svc,6.166903e-08,True
proposed vs xgb,3.018658e-04,True
proposed vs gnb,2.925433e-03,True
proposed vs rfc,1.572992e-01,False
